# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---

## Step 1: Dataset Exploration

Visualize the German Traffic Signs Dataset. This is open ended, some suggestions include: plotting traffic signs images, plotting the count of each sign, etc. Be creative!


The pickled data is a dictionary with 4 key/value pairs:

- features -> the images pixel values, (width, height, channels)
- labels -> the label of the traffic sign
- sizes -> the original width and height of the image, (width, height)
- coords -> coordinates of a bounding box around the sign in the image, (x1, y1, x2, y2). Based the original image (not the resized version).

In [2]:
%matplotlib inline
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import cv2
import matplotlib.pylab as plt
import matplotlib.image as mpimg
from sklearn.preprocessing import LabelBinarizer, MinMaxScaler
from traffic_sign_bencmark import *
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [3]:
# TODO: fill this in based on where you saved the training and testing data
training_file = 'train.p'
testing_file = 'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [4]:
### To start off let's do a basic data summary.

# TODO: number of training examples
n_train = len(X_train)

# TODO: number of testing examples
n_test = len(X_test)

# TODO: what's the shape of an image?
image_shape = X_train[0].shape

# TODO: how many classes are in the dataset
n_classes = np.unique(y_train).shape[0]

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples = 39209
Number of testing examples = 12630
Image data shape = (32, 32, 3)
Number of classes = 43


In [5]:
def bind_image(feed, i):
    features = feed['features']
    (y,x) = feed['sizes'][i]
    x1,y1,x2,y2 = feed['coords'][i]
    
    resized = cv2.resize(features[i],dsize=(y,x))
    roi = resized[y1:y2, x1:x2]
    if roi.shape[1] != 0:
        resized_roi = cv2.resize(roi,dsize=(48,48))
    else:
        resized_roi = cv2.resize(resized,dsize=(48,48))
    return resized_roi

X_train = np.stack([bind_image(train, i) for i in range(len(X_train))])


In [6]:
X_train = np.array(X_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32)

In [ ]:
def normalize(image):
    return cv2.normalize(image, None, 0.0, 1.0, cv2.NORM_MINMAX)
X_train = np.array([normalize(image) for image in X_train])

In [ ]:
datagen = ImageDataGenerator(
    # featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    )
datagen.fit(X_train)
for X_batch, Y_batch in datagen.flow(X_train, y_train, batch_size=5):
    print(X_batch.shape)
    for i in range(0, 5):
        plt.subplot(330 + 1 + i)
        plt.imshow(X_batch[i])
    break

In [ ]:
### Data exploration visualization goes here.
### Feel free to use as many code cells as needed.

In [ ]:
def concat_nx_images(imga, imgb):
    """
    Combines two color image ndarrays side-by-side.
    """
    ha,wa = imga.shape[:2]
    hb,wb = imgb.shape[:2]
    max_height = np.max([ha,hb])
    total_width = wa+wb
    new_img = np.zeros(shape=(max_height, total_width, 3), dtype=np.uint8)
    new_img[:ha,:wa]=imga
    new_img[:hb,wa:wa+wb]=imgb
    return new_img


def concat_ny_images(imga, imgb):
    ha,wa = imga.shape[:2]
    hb,wb = imgb.shape[:2]
    total_height = ha+hb
    max_width = np.max([wa,wb])
    new_img = np.zeros(shape=(total_height, max_width, 3), dtype=np.uint8)
    new_img[:ha,:wa]=imga
    new_img[ha:ha+hb,:wa]=imgb
    return new_img

def concat_n_images(images):
    """
    """
    output = None
    for i, img in enumerate(images):
        if i==0:
            output = img
        else:
            output = concat_nx_images(output, img)
    return output

def display_images(X_train, j,k,l):
    final = None
    for i in range(j,k,l):
        output = concat_n_images(X_train[i:i+15])
        if i==0:
            final = output
        else:
            final = concat_ny_images(final,output)

    plt.imshow(final)
    plt.show()
display_images(X_train,0,150,15)

In [ ]:
def bind_image(feed, i):
    features = feed['features']
    (y,x) = feed['sizes'][i]
    x1,y1,x2,y2 = feed['coords'][i]
    
    resized = cv2.resize(features[i],dsize=(y,x))
    roi = resized[y1:y2, x1:x2]
    if roi.shape[1] != 0:
        resized_roi = cv2.resize(roi,dsize=(48,48))
    else:
        resized_roi = cv2.resize(resized,dsize=(48,48))
    return resized_roi

bound = np.stack([bind_image(train, i) for i in range(150)])
display_images(bound,0,150,15)

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Your model can be derived from a deep feedforward net or a deep convolutional network.
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
run_benchmark(X_train, y_train, X_test, y_test)

In [ ]:
bound_train = np.stack([bind_image(train, i) for i in range(len(X_train))])

In [ ]:
bound_test = np.stack([bind_image(test, i) for i in range(len(X_test))])

In [ ]:
run_benchmark(bound_train, y_train, bound_test, y_test)

In [ ]:
### Preprocess the data here.
### Feel free to use as many code cells as needed.


### Question 1 

_Describe the techniques used to preprocess the data._

**Answer:**

In [ ]:
### Generate data additional (if you want to!)
### and split the data into training/validation/testing sets here.
### Feel free to use as many code cells as needed.

### Question 2

_Describe how you set up the training, validation and testing data for your model. If you generated additional data, why?_

**Answer:**

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

### Question 3

_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)  For reference on how to build a deep neural network using TensorFlow, see [Deep Neural Network in TensorFlow
](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/83a3a2a2-a9bd-4b7b-95b0-eb924ab14432) from the classroom._


**Answer:**

In [ ]:
### Train your model here.
### Feel free to use as many code cells as needed.

### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_


**Answer:**

### Question 5


_What approach did you take in coming up with a solution to this problem?_

**Answer:**

---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It would be helpful to plot the images in the notebook._



**Answer:**

In [ ]:
### Run the predictions here.
### Feel free to use as many code cells as needed.

### Question 7

_Is your model able to perform equally well on captured pictures or a live camera stream when compared to testing on the dataset?_


**Answer:**

In [ ]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.

### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.11/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*


**Answer:**

### Question 9
_If necessary, provide documentation for how an interface was built for your model to load and classify newly-acquired images._


**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.